# Equivalence of several methods of state evolution by the same unitary #
1. state vector evolution
2. density matrix evolution
3. superoperator evolution

In [1]:
import numpy as np
from qiskit.quantum_info import random_statevector, random_unitary

In [2]:
n_qubits = 1
dim = 2**n_qubits

### Generate a random unitary and state vector ###

In [3]:
U = random_unitary(dim).data
psi = random_statevector(dim).data.reshape(-1, 1)
psi_dm = psi @ psi.conj().T

## 1. state vector evolution ##
$$ U |\psi> = |\phi> $$
And the density matrix for $|\phi>$ is: 
$$|\phi><\phi|$$

In [4]:
phi = U @ psi
phi_dm1 = phi @ phi.conj().T

## 2. density matrix evolution
$$ U |\phi><\phi| U^\dagger = |\phi><\phi| $$

In [5]:
phi_dm2 = U @ psi_dm @ U.conj().T

## 3. superoperator evolution ##

Using the "[vec trick](https://en.wikipedia.org/wiki/Kronecker_product#Matrix_equations)" we can start from density matrix evolution and get:
$$
\text{vec}(U |\psi><\psi| U^\dagger) = (U^* \otimes U) \text{vec}(|\psi><\psi|)
$$

Where $(U^* \otimes U)$ represents a superoperator and we define $\text{vec}()$ to be the function that vectorizes a matrix by stacking columns in column-order. We can also define $\text{vec}^{-1}()$ to be the function that undoes the $\text{vec}()$ operation and converts back to a density matrix.
$$ \text{vec}^{-1}((U^* \otimes U) \text{vec}(|\psi><\psi|)) = |\phi><\phi|$$ 

In [6]:
vec = lambda X : X.reshape(-1, 1, order='F')
vec_inverse = lambda X : X.reshape(dim, dim, order='F')

In [7]:
superop = np.kron(U.conj(), U)
phi_dm3 = vec_inverse(superop @ vec(psi_dm))

Note that if instead we define $\text{vec}_{row}()$ such that it vectorizes by stacking columns in row-order then it changes to the following:
$$
\text{vec}_{row}(U |\psi><\psi| U^\dagger) = (U \otimes U^*) \text{vec}_{row}(|\psi><\psi|)\\
\text{vec}_{row}^{-1}((U^* \otimes U) \text{vec}_{row}(|\psi><\psi|)) = |\phi><\phi| 
$$

In [8]:
vec_row = lambda X : X.reshape(-1, 1)
vec_row_inverse = lambda X : X.reshape(dim, dim)

In [9]:
superop = np.kron(U, U.conj())
phi_dm4 = vec_row_inverse(superop @ vec_row(psi_dm))

## Demonstration of equivalence ##

In [10]:
print(np.allclose(phi_dm1, phi_dm2, phi_dm3, phi_dm4))

True


In [11]:
print("psi:\n", psi, "\n")
print("U:\n", U, "\n")
print("phi_dm:\n", phi_dm1)

psi:
 [[ 0.65545951+0.59174038j]
 [-0.33760827-0.32593989j]] 

U:
 [[-0.41329769-0.37538345j -0.64273189-0.52456458j]
 [-0.77192535-0.30397951j  0.52957853+0.17684539j]] 

phi_dm:
 [[0.01082864+7.34583059e-21j 0.09364129+4.40759749e-02j]
 [0.09364129-4.40759749e-02j 0.98917136+1.36522017e-18j]]
